In [242]:
import gymnasium as gym
import ale_py
import cv2
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import random
from collections import deque
from tqdm import tqdm
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime

In [243]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device

device(type='cuda')

In [244]:
gym.register_envs(ale_py)
current_time = datetime.now().strftime("%Y-%m-%d_%H-%M-%S")
writer = SummaryWriter(f"runs/pong_dqn_{current_time}")

In [245]:
BATCH_SIZE = 32
GAMMA = 0.99
EPSILON_START = 1.0
EPSILON_END = 0.02
EPSILON_DECAY = 1000000
TARGET_UPDATE = 1000
MEMORY_SIZE = 4
LEARNING_RATE = 1e-4

In [246]:
def preprocess_observation(obs):
    obs = obs[35:195]  # Crop
    obs = cv2.resize(obs, (84, 84))  # Resize
    obs = cv2.cvtColor(obs, cv2.COLOR_RGB2GRAY)  # Convert to grayscale
    _, obs = cv2.threshold(obs, 1, 255, cv2.THRESH_BINARY)  # Binary
    return obs / 255.0  # Normalize


In [247]:
# Define the DQN model
class DQN(nn.Module):
    def __init__(self, action_space):
        super(DQN, self).__init__()
        self.conv1 = nn.Conv2d(4, 32, kernel_size=8, stride=4)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=4, stride=2)
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3, stride=1)
        self.fc = nn.Linear(7 * 7 * 64, 512)
        self.out = nn.Linear(512, action_space)

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = torch.relu(self.conv2(x))
        x = torch.relu(self.conv3(x))
        x = x.view(x.size(0), -1)
        x = torch.relu(self.fc(x))
        return self.out(x)

In [248]:
memory = deque(maxlen=MEMORY_SIZE)

In [249]:
def select_action(state, epsilon, action_space):
    if random.random() < epsilon:
        return random.randrange(action_space)
    else:
        if isinstance(state, np.ndarray):
            state = torch.tensor(state, dtype=torch.float32).unsqueeze(0)
        with torch.no_grad():
            return torch.argmax(policy_net(state)).item()

In [250]:
def optimize_model():
    if len(memory) < BATCH_SIZE:
        return

    transitions = random.sample(memory, BATCH_SIZE)
    batch_state, batch_action, batch_reward, batch_next_state, batch_done = zip(*transitions)

    # Conversion en tenseurs PyTorch
    batch_state = torch.cat(batch_state).unflatten(0,(BATCH_SIZE,-1)).to(device)
    batch_action = torch.tensor(batch_action).to(device)
    batch_reward = torch.tensor(batch_reward).to(device)
    batch_next_state = torch.cat(batch_next_state).unflatten(0,(BATCH_SIZE,-1)).to(device)
    batch_done = torch.tensor(batch_done, dtype=torch.bool).to(device)

    current_q_values = policy_net(batch_state).gather(1, batch_action.unsqueeze(1)).to(device)
    next_q_values = target_net(batch_next_state).max(1)[0].detach().to(device)
    expected_q_values = batch_reward + (GAMMA * next_q_values) * (~batch_done).to(device)

    loss = nn.SmoothL1Loss()(current_q_values, expected_q_values.unsqueeze(1))
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss.item()



In [251]:
env = gym.make("PongNoFrameskip-v4", difficulty=1)
policy_net = DQN(env.action_space.n).to(device)
target_net = DQN(env.action_space.n).to(device)
target_net.load_state_dict(policy_net.state_dict())
target_net.eval()

optimizer = optim.Adam(policy_net.parameters(), lr=LEARNING_RATE)

In [252]:
num_episodes = 5000
epsilon = EPSILON_START

global_step = 0

for episode in tqdm(range(num_episodes), desc="Training Episodes"):
    obs, _ = env.reset()
    state = preprocess_observation(obs)
    state = np.stack([state] * 4, axis=0)
    state = torch.tensor(state, dtype=torch.float32).to(device)

    total_reward = 0
    done = False
    steps = 0

    while not done:
        action = select_action(state, epsilon, env.action_space.n)
        next_obs, reward, done, truncated, _ = env.step(action)
        done = done or truncated
        print(done)
        total_reward += reward
        next_state = preprocess_observation(next_obs)
        next_state = np.concatenate((state[1:, :, :].cpu().numpy(),
                                     np.expand_dims(next_state, 0)), axis=0)
        next_state = torch.tensor(next_state, dtype=torch.float32).to(device)
        memory.append((state, action, reward, next_state, done))
        state = next_state

        # Optimisation du modèle et enregistrement de la perte
        loss = optimize_model()
        if loss is not None:
            writer.add_scalar("Loss", loss, global_step)
        
        if steps % TARGET_UPDATE == 0:
            target_net.load_state_dict(policy_net.state_dict())

        steps += 1
        global_step += 1  # Incrémente global_step pour chaque étape
        epsilon = max(EPSILON_END, EPSILON_START - steps / EPSILON_DECAY)
        
        for name, param in policy_net.named_parameters():
          if param.requires_grad:
              writer.add_histogram(f"{name}_weights", param, episode)
              if param.grad is not None:
                  writer.add_histogram(f"{name}_gradients", param.grad, episode)
        
        if reward > 0:
          print(f"Reward: {reward}")
        
        writer.add_scalar("Epsilon", epsilon, global_step)

    writer.add_scalar("Total Reward", total_reward, episode)
    tqdm.write(f"Episode {episode}, Total Reward: {total_reward}")

writer.close()  # Ferme le writer après l'entraînement

Training Episodes:   0%|          | 0/5000 [00:00<?, ?it/s]

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

Training Episodes:   0%|          | 0/5000 [03:01<?, ?it/s]

False


RuntimeError: mat1 and mat2 shapes cannot be multiplied (64x49 and 3136x512)